In [14]:
import os
from glob import glob
from typing import List, Tuple, Optional
from pathlib import Path

import uuid
import json

source_base_url = 'https://doc.tg-cloud.co.kr/manual/console/'
image_base_url = 'https://doc.tg-cloud.co.kr/manual/console/'

In [15]:
def scan_directory(root_path: str) -> List[str]:
    """
    디렉토리를 스캔하여 .md 파일 경로를 찾습니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[str]: 발견된 .md 파일의 절대 경로 리스트
    
    Example:
        >>> scan_directory("./manual/user/firstUser")
        ['/path/to/manual/user/firstUser/login/login.md', 
         '/path/to/manual/user/firstUser/approval/approval.md']
    """
    md_files = []
    
    if not os.path.exists(root_path):
        print(f"⚠️ 경로가 존재하지 않습니다: {root_path}")
        return md_files
    
    for root, dirs, files in os.walk(root_path):
        #print(root,dirs,files)
        
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                md_files.append(os.path.abspath(file_path))
    
    return sorted(md_files)


In [16]:
def clean_markdown_text(text):
    # 마크다운 문법 제거
    text = re.sub(r'[#*`]', '', text)  # #, *, ` 제거
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)  # 링크 텍스트만 유지
    return text

def normalize_whitespace(text):
    # 연속된 공백을 하나로 통일, 줄 시작/끝 공백 제거
    return ' '.join(text.split())

def extract_plain_text(text):
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    # 특수문자 제거 (단, 한글/영문/숫자/기본 문장부호는 유지)
    text = re.sub(r'[^\w\s\.,!?-]', '', text)
    return text

def remove_empty_lines(text):
    # 빈 줄 제거
    return '\n'.join(line for line in text.split('\n') if line.strip())


In [17]:
import re
from typing import List, Dict


def parse_markdown(manual_type: str, menu_type: str, source_url: str, image_url_base: str, markdown_filepath: str, isJson: bool = False) -> List[Dict[str, any]]:
    """마크다운 파일을 파싱하여 섹션별 데이터를 추출하는 함수
    
    Args:
        manual_type (str): 매뉴얼 타입
        user_type (str): 사용자 타입 
        source_url (str): 소스 URL
        image_url_base (str): 이미지 URL 기본 경로
        markdown_filepath (str): 마크다운 파일 경로
        isJson (bool, optional): JSON 파일 저장 여부. Defaults to False.
        
    Returns:
        List[Dict[str, any]]: 파싱된 섹션 데이터 리스트
    """
    with open(markdown_filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sections = []
    current_section = None
    image_urls = []
    in_toc = False

    for line in lines:
        # Check for the start of a new section
        if line.startswith('##') and not line.startswith('## **목차**'):
            if current_section:
                sections.append({
                    'manual_type': manual_type,
                    'menu_type': menu_type,
                    'section': current_section.strip(),
                    'source_url': source_url,
                    'image_urls': image_urls
                })
            current_section = line.strip('# ').strip()
            image_urls = []
            in_toc = False
        elif line.startswith('## **목차**'):
            in_toc = True
        elif in_toc:
            continue
        elif current_section is not None:
            # Check for image links
            image_match = re.findall(r'!\[.*?\]\((img/[^)]+)\)', line)
            for img in image_match:
                image_urls.append(f'{image_url_base}{img}')
            #current_section += line
            # Only add non-image lines to the section
            
            if not image_match:
                current_section += line
    # Add the last section if it exists
    if current_section:
        sections.append({
            'manual_type': manual_type,
            'menu_type': menu_type,
            'section': current_section.strip(),
            'source_url': source_url,
            'image_urls': image_urls
        })

    # 데이터 정제 수행
    for section in sections:
        # 각 섹션의 'section' 키의 값을 순차적으로 정제
        section['section'] = clean_markdown_text(section['section'])  # 마크다운 텍스트 정제
        section['section'] = normalize_whitespace(section['section'])  # 공백 정규화
        section['section'] = extract_plain_text(section['section'])   # 일반 텍스트 추출
        section['section'] = remove_empty_lines(section['section'])   # 빈 줄 제거
    
    # JSON 파일로 저장
    if isJson:
        json_filepath = markdown_filepath.replace('.md', '.json')
        with open(json_filepath, 'w', encoding='utf-8') as f:
            json.dump(sections, f, ensure_ascii=False, indent=2)
            
    return sections

In [ ]:
# path = '/Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/approval/approval.md'
# 해당 패스중 마지막 5개 요소를 추출하여 리스트로 반환
def extract_path_components(path)->List:
        # 경로를 '/' 기준으로 분리
        components = path.split('/')
        
        # 뒤에서부터 순서대로 출력
        # 뒤에서부터 5개 요소만 추출
        result = []
        start_idx = max(len(components) - 5, 0)  # 음수 인덱스 방지
        for i in range(len(components)-1, start_idx-1, -1):
            result.append(components[i])

        return result


In [29]:
test_path = "../manual"

md_files = scan_directory(test_path)
print(f"발견된 파일 수: {len(md_files)}")

for i, filepath in enumerate(md_files, 1):
    print(f"[{i}] {filepath}")

발견된 파일 수: 48
[1] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/approval/approval.md
[2] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/bookmark/bookmark.md
[3] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/computing/cluster.md
[4] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/computing/node.md
[5] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/configs/configmaps.md
[6] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/configs/hpa.md
[7] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/configs/limitranges.md
[8] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/configs/priorityclasses.md
[9] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/configs/resourcequotas.md
[10] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/configs/secrets.md
[11] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/dashboard/dashboard.md
[12] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/firstUser/approval/approval.md
[13] /Users/gu.han/Documents/AI.WORK/RAG_Master/manual/firstUser/logi

In [30]:

if md_files:
    print("발견된 모든 파일들:")
    for i, markdown_filepath in enumerate(md_files, 1):
        
        result = extract_path_components(markdown_filepath)
        print(result)

       # markdown_filepath = './manual/user/firstUser/namespaces/namespaces.md'
        manual_type = result[2]
        menu_type =  result[1]
        
        # source_base_url = 'https://doc.tg-cloud.co.kr/manual/console/'
        # image_base_url = 'https://doc.tg-cloud.co.kr/manual/console/'

        source_url = source_base_url + manual_type + '/' + menu_type + '/' + menu_type
        image_url = image_base_url + manual_type + '/' + menu_type + '/'

        # 마크다운 파싱하여 데이터 추출
        data_dicts = parse_markdown(manual_type, menu_type, source_url, image_url, markdown_filepath,True)
        
        
else:
    print("발견된 파일이 없습니다.")

            

발견된 모든 파일들:
['approval.md', 'approval', 'manual', 'RAG_Master', 'AI.WORK']
['bookmark.md', 'bookmark', 'manual', 'RAG_Master', 'AI.WORK']
['cluster.md', 'computing', 'manual', 'RAG_Master', 'AI.WORK']
['node.md', 'computing', 'manual', 'RAG_Master', 'AI.WORK']
['configmaps.md', 'configs', 'manual', 'RAG_Master', 'AI.WORK']
['hpa.md', 'configs', 'manual', 'RAG_Master', 'AI.WORK']
['limitranges.md', 'configs', 'manual', 'RAG_Master', 'AI.WORK']
['priorityclasses.md', 'configs', 'manual', 'RAG_Master', 'AI.WORK']
['resourcequotas.md', 'configs', 'manual', 'RAG_Master', 'AI.WORK']
['secrets.md', 'configs', 'manual', 'RAG_Master', 'AI.WORK']
['dashboard.md', 'dashboard', 'manual', 'RAG_Master', 'AI.WORK']
['approval.md', 'approval', 'firstUser', 'manual', 'RAG_Master']
['login.md', 'login', 'firstUser', 'manual', 'RAG_Master']
['namespaces.md', 'namespaces', 'firstUser', 'manual', 'RAG_Master']
['project.md', 'project', 'firstUser', 'manual', 'RAG_Master']
['header.md', 'header', 'manual', 